# Data Cleaning & Extraction

In [2]:
import sqlite3
import csv
import pandas as pd

### Data Cleaning and required columns extraction for the initial .csv files from Kaggle

Dataset link : https://www.kaggle.com/datasets/rohitsahoo/bicycle-store-dataset?select=Transactions.csv

In [5]:
# Import the dataset
dataset_2 = pd.read_csv('Customer_ Demographic.csv', encoding='latin-1')

# Replace all empty values in the entire DataFrame with NULL
# dataset_2.replace('', None, inplace=True)

# Save cleaned data
file_name_2 = 'Cleaned_Customer_Demographic.csv'
dataset_2.to_csv(file_name_2, index=False)

# Specify the the required columns as a list and create a new cleaned .csv file
selected_columns_2 = dataset_2[['customer_id', 'first_name', 'last_name', 'gender', 'past_3_years_bike_related_purchases', 'DOB', 'job_title', 'job_industry_category', 'wealth_segment']]
selected_columns_2.to_csv(file_name_2, index=False)

In [6]:
# Import the dataset
dataset_3 = pd.read_csv('Customer_List.csv')

# Replace empty values with NULL
# dataset_3.replace('', None, inplace=True)

# Save cleaned data
file_name_3 = 'Cleaned_Customer_List.csv'
dataset_3.to_csv(file_name_3, index=False)

# Specify the the required columns as a list and create a new cleaned .csv file
selected_columns_3 = dataset_3[['first_name', 'last_name', 'gender', 'past_3_years_bike_related_purchases', 'DOB', 'job_title', 'job_industry_category', 'wealth_segment', 'owns_car']]
selected_columns_3.to_csv(file_name_3, index=False)


In [7]:
# Import the dataset
dataset_4 = pd.read_csv('Transactions.csv')

# Replace empty values with NULL
# dataset_4.replace('', None, inplace=True)

# Save cleaned data
file_name_4 = 'Cleaned_Transactions.csv'
dataset_4.to_csv(file_name_4, index=False)

# Specify the the required columns as a list and create a new cleaned .csv file
selected_columns_4 = dataset_4[['transaction_id', 'product_id', 'customer_id', 'transaction_date', 'online_order', 'order_status', 'brand', 'product_line', 'product_class', 'product_size']]
selected_columns_4.to_csv(file_name_4, index=False)


### Load .csv files and save as tables in the database

In [8]:
# Connect to the SQLite database
conn = sqlite3.connect('Bicycle_store.db')
cursor = conn.cursor()

In [9]:
# Dictionary containing information about each CSV file and its corresponding table structure
csv_tables = {

    'Cleaned_Customer_Demographic.csv': {
        'table_name': 'Customer_Demographic',
        'columns': [
            'customer_id INTEGER PRIMARY KEY',
            'first_name TEXT',
            'last_name TEXT',
            'gender TEXT',
            'past_3_years_bike_related_purchases INTEGER',
            'DOB DATE',
            'job_title TEXT',
            'job_industry_category TEXT',
            'wealth_segment TEXT',
        ]
    },
    'Cleaned_Customer_List.csv': {
        'table_name': 'Customer_List',
        'columns': [
            'first_name TEXT',
            'last_name TEXT',
            'gender TEXT',
            'past_3_years_bike_related_purchases INTEGER',
            'DOB DATE',
            'job_title TEXT',
            'job_industry_category TEXT',
            'wealth_segment TEXT',
            'owns_car TEXT'
        ]
    },
    'Cleaned_Transactions.csv': {
        'table_name': 'Transactions',
        'columns': [
            'transaction_id INTEGER PRIMARY KEY',
            'product_id INTEGER',
            'customer_id INTEGER',
            'transaction_date DATE',
            'online_order TEXT',
            'order_status TEXT',
            'brand TEXT',
            'product_line TEXT',
            'product_class TEXT',
            'product_size TEXT'
        ]
    }
}

# Create tables and insert data from CSV files
for csv_filename, table_info in csv_tables.items():
    table_name = table_info['table_name']
    columns = ', '.join(table_info['columns'])

    # Create table
    cursor.execute(f'CREATE TABLE IF NOT EXISTS {table_name} ({columns})')

    # Read CSV and insert data into the table
    with open(csv_filename, 'r', newline='') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # Skip header row
        for row in csv_reader:
            row = [value if value != '' else None for value in row]
            placeholders = ', '.join(['?'] * len(row))
            insert_query = f'INSERT INTO {table_name} VALUES ({placeholders})'
            cursor.execute(insert_query, row)


### Perform Queries within database(SQL)


In [10]:
# Q1
COMMAND = """SELECT Customer_Demographic.first_name FROM Customer_Demographic LEFT JOIN Transactions ON Customer_Demographic.customer_id = Transactions.customer_id
WHERE Transactions.online_order = 'True'
"""
data = pd.read_sql_query(COMMAND, conn)


In [11]:
data.head()

,first_name
0,Lauree
1,Norma
2,Damien
3,Sanders
4,Maynord


In [74]:
# Q2
COMMAND = """SELECT COUNT(*) AS 'missing value' FROM Customer_Demographic WHERE Customer_Demographic.last_name IS NULL"""

data = pd.read_sql_query(COMMAND, conn)

In [75]:
data

,missing value
0,125


In [14]:
# Q3
COMMAND = """SELECT DISTINCT Customer_Demographic.first_name, Customer_Demographic.last_name
FROM Customer_Demographic
WHERE Customer_Demographic.customer_id NOT IN (
    SELECT DISTINCT Transactions.customer_id
    FROM Transactions
    WHERE Transactions.transaction_date BETWEEN '2017-03-23' AND '2017-09-21')"""

data = pd.read_sql_query(COMMAND, conn)

In [15]:
data.head()

,first_name,last_name
0,Laraine,Medendorp
1,Eli,Bockman
2,Arlin,Dearle
3,Talbot,None
4,Sheila-kathryn,Calton


In [16]:
# Q4
COMMAND = """SELECT Customer_List.first_name, Customer_List.past_3_years_bike_related_purchases FROM Customer_List WHERE Customer_List. past_3_years_bike_related_purchases > 50"""
data = pd.read_sql_query(COMMAND, conn)


In [17]:
data.head()

,first_name,past_3_years_bike_related_purchases
0,Chickie,86.0
1,Morly,69.0
2,Lucine,64.0
3,Nancie,74.0
4,Barthel,72.0


In [18]:
# Q5
COMMAND = """SELECT Customer_Demographic.wealth_segment, COUNT(*) AS record_count FROM Customer_Demographic GROUP BY Customer_Demographic.wealth_segment"""
data = pd.read_sql_query(COMMAND, conn)

In [19]:
data

,wealth_segment,record_count
0,Affluent Customer,979
1,High Net Worth,1021
2,Mass Customer,2000


In [20]:
# Q6
COMMAND = """SELECT * FROM Customer_List WHERE Customer_List.past_3_years_bike_related_purchases = (SELECT MIN(past_3_years_bike_related_purchases) FROM Customer_List) OR Customer_List.past_3_years_bike_related_purchases  = (SELECT MAX(past_3_years_bike_related_purchases) FROM Customer_List)
"""
data = pd.read_sql_query(COMMAND, conn)

In [21]:
data.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,owns_car
0,Tobias,Woodhams,Male,0.0,15-04-1961,Research Nurse,Health,Mass Customer,No
1,Meriel,Tapp,Female,0.0,13-08-1995,VP Sales,IT,Mass Customer,No
2,Regine,Bownes,Female,99.0,01-07-1952,Senior Developer,Retail,Affluent Customer,No
3,Hersh,Stubbert,Male,0.0,25-06-2000,Technical Writer,Manufacturing,Mass Customer,Yes
4,Theresa,Cowper,Female,99.0,24-08-1976,Accountant III,Manufacturing,Mass Customer,No


In [22]:
# Q7
COMMAND = """SELECT t1.customer_id FROM Transactions t1 INNER JOIN Transactions t2 ON t1.customer_id = t2.transaction_id"""

data = pd.read_sql_query(COMMAND, conn)

In [23]:
data.head()

,customer_id
0,2950
1,3120
2,402
3,3135
4,787


### Perform Queries within database(Pandas DataFrame)

In [30]:
df_1 = pd.read_csv('Cleaned_Customer_Demographic.csv')
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   customer_id                          4000 non-null   int64 
 1   first_name                           4000 non-null   object
 2   last_name                            3875 non-null   object
 3   gender                               4000 non-null   object
 4   past_3_years_bike_related_purchases  4000 non-null   int64 
 5   DOB                                  3913 non-null   object
 6   job_title                            3494 non-null   object
 7   job_industry_category                3344 non-null   object
 8   wealth_segment                       4000 non-null   object
dtypes: int64(2), object(7)
memory usage: 281.4+ KB


In [31]:
df_2 = pd.read_csv('Cleaned_Customer_List.csv')
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   first_name                           1000 non-null   object
 1   last_name                            971 non-null    object
 2   gender                               1000 non-null   object
 3   past_3_years_bike_related_purchases  1000 non-null   int64 
 4   DOB                                  983 non-null    object
 5   job_title                            894 non-null    object
 6   job_industry_category                835 non-null    object
 7   wealth_segment                       1000 non-null   object
 8   owns_car                             1000 non-null   object
dtypes: int64(1), object(8)
memory usage: 70.4+ KB


In [32]:
df_3 = pd.read_csv('Cleaned_Transactions.csv')
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    20000 non-null  int64 
 1   product_id        20000 non-null  int64 
 2   customer_id       20000 non-null  int64 
 3   transaction_date  20000 non-null  object
 4   online_order      19640 non-null  object
 5   order_status      20000 non-null  object
 6   brand             19803 non-null  object
 7   product_line      19803 non-null  object
 8   product_class     19803 non-null  object
 9   product_size      19803 non-null  object
dtypes: int64(3), object(7)
memory usage: 1.5+ MB


In [71]:
# Q1 - Return the customer's first name who made the online order.
result = pd.merge(df_1, df_3, on='customer_id', how='left')
result = result[result['online_order'] == True]
customer_first_names = result['first_name'].drop_duplicates()

display(customer_first_names.head())

1            Laraine
11               Eli
16             Arlin
23            Talbot
25    Sheila-kathryn
Name: first_name, dtype: object

In [66]:
# Q2 - How many missing values are there in the 'last name' column in the 'Customer_Demographic' table?
count_missing_values = df_1['last_name'].isnull().sum()

print("count_missing_value : " , count_missing_values)

count_missing_value :  125


In [64]:
# Q3 - Return the name of the customers who did not place orders from 2017-03-23 to 2017-09-21
customer_ordered = df_3[(df_3['transaction_date'] >= '2017-03-23') & (df_3['transaction_date'] <= '2017-09-21')]['customer_id'].unique()
customer_did_not_order = df_1[~df_1['customer_id'].isin(customer_ordered)]

display(customer_did_not_order[['first_name', 'last_name']].head())

,first_name,last_name
0,Laraine,Medendorp
1,Eli,Bockman
2,Arlin,Dearle
3,Talbot,NaN
4,Sheila-kathryn,Calton


In [63]:
# Q4 - Return the name of the customer who made bike related purchase more than 50 over past 3 years
customer_bike_purchase = df_2[df_2['past_3_years_bike_related_purchases'] > 50]

display(customer_bike_purchase['first_name'].head())

0    Chickie
1      Morly
3     Lucine
7     Nancie
9    Barthel
Name: first_name, dtype: object

In [62]:
# Q5 - Categorize and count records by ‘wealth_segment’ in Customer_Demographic table
result = df_1.groupby('wealth_segment').size().reset_index(name='record_count')

display(result)

,wealth_segment,record_count
0,Affluent Customer,979
1,High Net Worth,1021
2,Mass Customer,2000


In [61]:
# Q6 - Return the Customer List that contains the minimum and maximum values of past_3_years_bike_related_purchases
min_purchase = df_2['past_3_years_bike_related_purchases'].min()
max_purchase = df_2['past_3_years_bike_related_purchases'].max()
result = df_2[(df_2['past_3_years_bike_related_purchases'] == min_purchase)|(df_2['past_3_years_bike_related_purchases'] == max_purchase)]

display(result.head())

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,owns_car
76,Tobias,Woodhams,Male,0,15-04-1961,Research Nurse,Health,Mass Customer,No
113,Meriel,Tapp,Female,0,13-08-1995,VP Sales,IT,Mass Customer,No
168,Regine,Bownes,Female,99,01-07-1952,Senior Developer,Retail,Affluent Customer,No
225,Hersh,Stubbert,Male,0,25-06-2000,Technical Writer,Manufacturing,Mass Customer,Yes
272,Theresa,Cowper,Female,99,24-08-1976,Accountant III,Manufacturing,Mass Customer,No


In [73]:
# Q7 - Return the customer id who has purchase history
# Inner join df_3('Transsaction') table
result = pd.merge(df_3, df_3, left_on='customer_id', right_on='transaction_id')
result = result.drop(columns=['transaction_id_y'])
customer_purchase_history = result['customer_id_x']

display(customer_purchase_history.drop_duplicates().head())

0     2950
3     3120
10     402
16    3135
23     787
Name: customer_id_x, dtype: int64

In [76]:
# Commit changes and close the database connection
conn.commit()
conn.close()